# Melting Point Prediction with Molecular Descriptors

This notebook generates molecular descriptors using RDKit and Mordred, creates visualizations, and trains XGBoost/LightGBM models.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, rdMolDescriptors
from mordred import Calculator, descriptors
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print("Libraries loaded successfully!")

## 1. Load Data

In [ ]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"\nTarget (Tm) statistics:")
print(train_df['Tm'].describe())

## 2. Generate Molecular Descriptors

We'll create descriptors that are closely tied to melting point:
- **Molecular Weight**: Larger molecules tend to have higher melting points
- **LogP**: Lipophilicity affects crystal packing
- **TPSA**: Polar surface area relates to intermolecular forces
- **H-bond donors/acceptors**: Hydrogen bonding increases melting point
- **Rotatable bonds**: Molecular flexibility (lower = higher Tm)
- **Aromatic rings**: Pi-stacking increases melting point
- **Fraction sp3**: Molecular shape affects packing

In [ ]:
def compute_rdkit_descriptors(smiles_list):
    """Compute RDKit descriptors related to melting point"""
    descriptors_list = []
    
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            descriptors_list.append({key: np.nan for key in [
                'MolWt', 'LogP', 'TPSA', 'NumHDonors', 'NumHAcceptors',
                'NumRotatableBonds', 'NumAromaticRings', 'NumHeavyAtoms',
                'FractionCSP3', 'NumRings', 'NumHeteroatoms', 'MolMR',
                'NumValenceElectrons', 'NumRadicalElectrons', 'HallKierAlpha',
                'NumAliphaticRings', 'NumSaturatedRings', 'NumAromaticHeterocycles',
                'NumAliphaticHeterocycles', 'NumSaturatedHeterocycles'
            ]})
            continue
            
        desc = {
            'MolWt': Descriptors.MolWt(mol),
            'LogP': Descriptors.MolLogP(mol),
            'TPSA': Descriptors.TPSA(mol),
            'NumHDonors': Lipinski.NumHDonors(mol),
            'NumHAcceptors': Lipinski.NumHAcceptors(mol),
            'NumRotatableBonds': Lipinski.NumRotatableBonds(mol),
            'NumAromaticRings': rdMolDescriptors.CalcNumAromaticRings(mol),
            'NumHeavyAtoms': Lipinski.HeavyAtomCount(mol),
            'FractionCSP3': rdMolDescriptors.CalcFractionCSP3(mol),
            'NumRings': rdMolDescriptors.CalcNumRings(mol),
            'NumHeteroatoms': rdMolDescriptors.CalcNumHeteroatoms(mol),
            'MolMR': Descriptors.MolMR(mol),  # Molar refractivity
            'NumValenceElectrons': Descriptors.NumValenceElectrons(mol),
            'NumRadicalElectrons': Descriptors.NumRadicalElectrons(mol),
            'HallKierAlpha': Descriptors.HallKierAlpha(mol),
            'NumAliphaticRings': rdMolDescriptors.CalcNumAliphaticRings(mol),
            'NumSaturatedRings': rdMolDescriptors.CalcNumSaturatedRings(mol),
            'NumAromaticHeterocycles': rdMolDescriptors.CalcNumAromaticHeterocycles(mol),
            'NumAliphaticHeterocycles': rdMolDescriptors.CalcNumAliphaticHeterocycles(mol),
            'NumSaturatedHeterocycles': rdMolDescriptors.CalcNumSaturatedHeterocycles(mol),
        }
        descriptors_list.append(desc)
    
    return pd.DataFrame(descriptors_list)

print("Computing RDKit descriptors for training data...")
train_rdkit = compute_rdkit_descriptors(train_df['SMILES'].values)
print("Computing RDKit descriptors for test data...")
test_rdkit = compute_rdkit_descriptors(test_df['SMILES'].values)

print(f"\nRDKit descriptors generated: {train_rdkit.shape[1]}")
print(train_rdkit.head())

In [ ]:
def compute_mordred_descriptors(smiles_list):
    """Compute Mordred descriptors that work for all molecules"""
    # Create molecules
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    
    # Use only 2D descriptors (no 3D) to ensure universal applicability
    calc = Calculator(descriptors, ignore_3D=True)
    
    print(f"Calculating {len(calc.descriptors)} Mordred descriptors...")
    df = calc.pandas(mols)
    
    # Convert to numeric and filter out columns with any errors/NaN
    df = df.apply(pd.to_numeric, errors='coerce')
    
    # Keep only columns with no NaN values
    valid_cols = df.columns[df.notna().all()].tolist()
    df_clean = df[valid_cols]
    
    # Remove columns with zero variance
    df_clean = df_clean.loc[:, df_clean.std() > 0]
    
    print(f"Valid Mordred descriptors (no NaN, non-zero variance): {df_clean.shape[1]}")
    return df_clean

print("Computing Mordred descriptors for training data...")
train_mordred = compute_mordred_descriptors(train_df['SMILES'].values)
print("\nComputing Mordred descriptors for test data...")
test_mordred = compute_mordred_descriptors(test_df['SMILES'].values)

# Ensure both have the same columns
common_mordred_cols = list(set(train_mordred.columns) & set(test_mordred.columns))
train_mordred = train_mordred[common_mordred_cols]
test_mordred = test_mordred[common_mordred_cols]

print(f"\nFinal Mordred descriptors: {len(common_mordred_cols)}")

## 3. Combine All Features

In [ ]:
# Original group features
group_cols = [col for col in train_df.columns if col.startswith('Group')]
train_groups = train_df[group_cols]
test_groups = test_df[group_cols]

# Combine all features
X_train_full = pd.concat([
    train_groups.reset_index(drop=True),
    train_rdkit.reset_index(drop=True),
    train_mordred.reset_index(drop=True)
], axis=1)

X_test_full = pd.concat([
    test_groups.reset_index(drop=True),
    test_rdkit.reset_index(drop=True),
    test_mordred.reset_index(drop=True)
], axis=1)

y_train = train_df['Tm']

print(f"Original group features: {len(group_cols)}")
print(f"RDKit descriptors: {train_rdkit.shape[1]}")
print(f"Mordred descriptors: {train_mordred.shape[1]}")
print(f"Total features: {X_train_full.shape[1]}")

## 4. Visualizations

### 4.1 Distribution of Melting Points

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(train_df['Tm'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(train_df['Tm'].mean(), color='red', linestyle='--', linewidth=2, label=f"Mean: {train_df['Tm'].mean():.1f} K")
axes[0].axvline(train_df['Tm'].median(), color='orange', linestyle='--', linewidth=2, label=f"Median: {train_df['Tm'].median():.1f} K")
axes[0].set_xlabel('Melting Point (K)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Melting Points', fontsize=14)
axes[0].legend()

# Box plot
axes[1].boxplot(train_df['Tm'], vert=True)
axes[1].set_ylabel('Melting Point (K)', fontsize=12)
axes[1].set_title('Melting Point Box Plot', fontsize=14)

plt.tight_layout()
plt.savefig('viz_1_melting_point_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: viz_1_melting_point_distribution.png")

### 4.2 Correlation Matrix of Key Descriptors with Melting Point

In [ ]:
# Select key RDKit descriptors for correlation analysis
key_descriptors = ['MolWt', 'LogP', 'TPSA', 'NumHDonors', 'NumHAcceptors', 
                   'NumRotatableBonds', 'NumAromaticRings', 'NumHeavyAtoms',
                   'FractionCSP3', 'NumRings', 'NumHeteroatoms', 'MolMR']

# Create correlation dataframe
corr_df = train_rdkit[key_descriptors].copy()
corr_df['Tm'] = train_df['Tm'].values

# Calculate correlation matrix
corr_matrix = corr_df.corr()

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, square=True, linewidths=0.5, ax=ax,
            cbar_kws={'label': 'Correlation Coefficient'})
ax.set_title('Correlation Matrix: Key Molecular Descriptors vs Melting Point', fontsize=14)

plt.tight_layout()
plt.savefig('viz_2_correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: viz_2_correlation_matrix.png")

# Print correlations with Tm
print("\nCorrelations with Melting Point (Tm):")
tm_corr = corr_matrix['Tm'].drop('Tm').sort_values(key=abs, ascending=False)
print(tm_corr)

### 4.3 Scatter Plots of Top Correlated Descriptors vs Melting Point

In [ ]:
# Top 4 descriptors by absolute correlation with Tm
top_descriptors = tm_corr.head(4).index.tolist()

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for i, desc in enumerate(top_descriptors):
    ax = axes[i]
    corr_val = tm_corr[desc]
    
    ax.scatter(train_rdkit[desc], train_df['Tm'], alpha=0.4, s=20, c='steelblue')
    
    # Add trend line
    z = np.polyfit(train_rdkit[desc].dropna(), train_df['Tm'].loc[train_rdkit[desc].notna()], 1)
    p = np.poly1d(z)
    x_line = np.linspace(train_rdkit[desc].min(), train_rdkit[desc].max(), 100)
    ax.plot(x_line, p(x_line), 'r--', linewidth=2, label=f'r = {corr_val:.3f}')
    
    ax.set_xlabel(desc, fontsize=12)
    ax.set_ylabel('Melting Point (K)', fontsize=12)
    ax.set_title(f'{desc} vs Melting Point', fontsize=13)
    ax.legend(loc='upper right')

plt.tight_layout()
plt.savefig('viz_3_scatter_top_descriptors.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: viz_3_scatter_top_descriptors.png")

### 4.4 Melting Point by Molecular Properties (Categorical Analysis)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Melting Point by Number of Aromatic Rings
ax = axes[0, 0]
ring_data = train_rdkit['NumAromaticRings'].copy()
ring_data = ring_data.clip(upper=5)  # Group 5+ together
ring_groups = pd.concat([ring_data, train_df['Tm']], axis=1)
ring_groups.boxplot(column='Tm', by='NumAromaticRings', ax=ax)
ax.set_xlabel('Number of Aromatic Rings', fontsize=12)
ax.set_ylabel('Melting Point (K)', fontsize=12)
ax.set_title('Melting Point by Aromatic Ring Count', fontsize=13)
plt.suptitle('')

# 2. Melting Point by H-bond Donors
ax = axes[0, 1]
hd_data = train_rdkit['NumHDonors'].copy()
hd_data = hd_data.clip(upper=4)
hd_groups = pd.concat([hd_data, train_df['Tm']], axis=1)
hd_groups.boxplot(column='Tm', by='NumHDonors', ax=ax)
ax.set_xlabel('Number of H-bond Donors', fontsize=12)
ax.set_ylabel('Melting Point (K)', fontsize=12)
ax.set_title('Melting Point by H-bond Donors', fontsize=13)
plt.suptitle('')

# 3. Melting Point by Molecular Weight bins
ax = axes[1, 0]
mw_bins = pd.cut(train_rdkit['MolWt'], bins=[0, 100, 200, 300, 400, 1000], 
                 labels=['<100', '100-200', '200-300', '300-400', '>400'])
mw_groups = pd.DataFrame({'MolWt_bin': mw_bins, 'Tm': train_df['Tm']})
mw_groups.boxplot(column='Tm', by='MolWt_bin', ax=ax)
ax.set_xlabel('Molecular Weight Range', fontsize=12)
ax.set_ylabel('Melting Point (K)', fontsize=12)
ax.set_title('Melting Point by Molecular Weight', fontsize=13)
plt.suptitle('')

# 4. Melting Point by Total Rings
ax = axes[1, 1]
ring_total = train_rdkit['NumRings'].copy()
ring_total = ring_total.clip(upper=5)
ring_total_groups = pd.concat([ring_total, train_df['Tm']], axis=1)
ring_total_groups.boxplot(column='Tm', by='NumRings', ax=ax)
ax.set_xlabel('Total Number of Rings', fontsize=12)
ax.set_ylabel('Melting Point (K)', fontsize=12)
ax.set_title('Melting Point by Total Ring Count', fontsize=13)
plt.suptitle('')

plt.tight_layout()
plt.savefig('viz_4_melting_point_by_properties.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: viz_4_melting_point_by_properties.png")

### 4.5 Feature Importance Heatmap (Top Mordred Descriptors)

In [ ]:
# Calculate correlations of Mordred descriptors with Tm
mordred_corr = train_mordred.corrwith(train_df['Tm']).dropna()
mordred_corr_sorted = mordred_corr.abs().sort_values(ascending=False)

# Top 20 Mordred descriptors by correlation
top_20_mordred = mordred_corr_sorted.head(20).index.tolist()
top_20_corr = mordred_corr[top_20_mordred]

# Create bar plot
fig, ax = plt.subplots(figsize=(12, 8))
colors = ['steelblue' if x > 0 else 'coral' for x in top_20_corr]
bars = ax.barh(range(len(top_20_corr)), top_20_corr.values, color=colors)
ax.set_yticks(range(len(top_20_corr)))
ax.set_yticklabels(top_20_corr.index)
ax.set_xlabel('Correlation with Melting Point', fontsize=12)
ax.set_title('Top 20 Mordred Descriptors by Correlation with Tm', fontsize=14)
ax.axvline(x=0, color='black', linewidth=0.5)
ax.invert_yaxis()

# Add correlation values on bars
for i, (bar, val) in enumerate(zip(bars, top_20_corr.values)):
    ax.text(val + 0.01 if val > 0 else val - 0.01, i, f'{val:.3f}', 
            va='center', ha='left' if val > 0 else 'right', fontsize=9)

plt.tight_layout()
plt.savefig('viz_5_top_mordred_correlations.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: viz_5_top_mordred_correlations.png")

## 5. Train Models with Enhanced Features

In [ ]:
# Handle any remaining NaN values
X_train_clean = X_train_full.fillna(0)
X_test_clean = X_test_full.fillna(0)

# Remove infinite values
X_train_clean = X_train_clean.replace([np.inf, -np.inf], 0)
X_test_clean = X_test_clean.replace([np.inf, -np.inf], 0)

print(f"Final training features: {X_train_clean.shape}")
print(f"Final test features: {X_test_clean.shape}")

In [ ]:
# Define models
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

lgbm_model = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_model(model, X, y, model_name):
    """Evaluate model using 5-fold cross-validation"""
    print(f"\n{'='*50}")
    print(f"Evaluating: {model_name}")
    print('='*50)
    
    y_pred_cv = cross_val_predict(model, X, y, cv=kfold)
    
    rmse = np.sqrt(mean_squared_error(y, y_pred_cv))
    mae = mean_absolute_error(y, y_pred_cv)
    r2 = r2_score(y, y_pred_cv)
    
    print(f"\nCross-Validation Results (5-Fold):")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    
    model.fit(X, y)
    
    return model, {'RMSE': rmse, 'MAE': mae, 'R2': r2}, y_pred_cv

In [ ]:
print("\n" + "="*60)
print("MODEL EVALUATION WITH ENHANCED FEATURES")
print("="*60)

xgb_trained, xgb_metrics, xgb_cv_pred = evaluate_model(xgb_model, X_train_clean, y_train, "XGBoost")
lgbm_trained, lgbm_metrics, lgbm_cv_pred = evaluate_model(lgbm_model, X_train_clean, y_train, "LightGBM")

In [ ]:
# Summary comparison
print("\n" + "="*60)
print("EVALUATION SUMMARY")
print("="*60)

results_df = pd.DataFrame({
    'Metric': ['RMSE', 'MAE', 'R²'],
    'XGBoost': [xgb_metrics['RMSE'], xgb_metrics['MAE'], xgb_metrics['R2']],
    'LightGBM': [lgbm_metrics['RMSE'], lgbm_metrics['MAE'], lgbm_metrics['R2']]
})

results_df['Best'] = results_df.apply(
    lambda row: 'XGBoost' if (row['Metric'] == 'R²' and row['XGBoost'] > row['LightGBM']) or 
                            (row['Metric'] != 'R²' and row['XGBoost'] < row['LightGBM'])
                else 'LightGBM', axis=1
)

print(results_df.to_string(index=False))

In [ ]:
# Generate predictions
xgb_predictions = xgb_trained.predict(X_test_clean)
lgbm_predictions = lgbm_trained.predict(X_test_clean)

# Save predictions
xgb_submission = pd.DataFrame({'id': test_df['id'], 'Tm': xgb_predictions})
lgbm_submission = pd.DataFrame({'id': test_df['id'], 'Tm': lgbm_predictions})

xgb_submission.to_csv('submission_xgboost_enhanced.csv', index=False)
lgbm_submission.to_csv('submission_lightgbm_enhanced.csv', index=False)

print("\nPredictions saved:")
print("  - submission_xgboost_enhanced.csv")
print("  - submission_lightgbm_enhanced.csv")

In [ ]:
# Plot actual vs predicted
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, pred, name, metrics in zip(axes, [xgb_cv_pred, lgbm_cv_pred], 
                                     ['XGBoost', 'LightGBM'],
                                     [xgb_metrics, lgbm_metrics]):
    ax.scatter(y_train, pred, alpha=0.4, s=20)
    ax.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', linewidth=2)
    ax.set_xlabel('Actual Melting Point (K)', fontsize=12)
    ax.set_ylabel('Predicted Melting Point (K)', fontsize=12)
    ax.set_title(f'{name}: Actual vs Predicted\nR² = {metrics["R2"]:.4f}, RMSE = {metrics["RMSE"]:.2f}', fontsize=13)

plt.tight_layout()
plt.savefig('viz_6_actual_vs_predicted.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: viz_6_actual_vs_predicted.png")

## 6. Feature Importance Analysis

In [ ]:
# Get feature importances from XGBoost
feature_importance = pd.DataFrame({
    'feature': X_train_clean.columns,
    'importance': xgb_trained.feature_importances_
}).sort_values('importance', ascending=False)

# Top 30 features
top_30 = feature_importance.head(30)

fig, ax = plt.subplots(figsize=(12, 10))
ax.barh(range(len(top_30)), top_30['importance'].values, color='steelblue')
ax.set_yticks(range(len(top_30)))
ax.set_yticklabels(top_30['feature'].values)
ax.set_xlabel('Feature Importance', fontsize=12)
ax.set_title('Top 30 Features by XGBoost Importance', fontsize=14)
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('viz_7_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: viz_7_feature_importance.png")

print("\nTop 20 Most Important Features:")
print(feature_importance.head(20).to_string(index=False))